In [1]:
# dataDB/chroma_scripts.ipynb  # load  < markdown + 图片  >
import os, hashlib, re, uuid 
from pathlib import Path
from typing import List, Union
from tqdm import tqdm

# 使用 Unstructured 更好地解析 Markdown 中的图片链接
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata # 过滤复杂元数据 chroma只支持: str int float bool None

In [2]:
# embedding
local_embedding_dir = r"../temp/mineru_models/Qwen3-Embedding-0.6B"
embedding_model = HuggingFaceEmbeddings(
    model_name=local_embedding_dir,
    model_kwargs={"trust_remote_code": True},
    encode_kwargs={"normalize_embeddings": True},
    cache_folder=None,  # 不使用额外缓存
)
print(f"✅ 模型加载完成: {embedding_model}")

✅ 模型加载完成: model_name='../temp/mineru_models/Qwen3-Embedding-0.6B' cache_folder=None model_kwargs={'trust_remote_code': True} encode_kwargs={'normalize_embeddings': True} query_encode_kwargs={} multi_process=False show_progress=False


In [3]:
markdown_file = r"..\docs.log\zhuanli_RobotHand\CN202021894937.5-一种结构紧凑的回转动力单元以及应用其的机器人\auto\CN202021894937.5-一种结构紧凑的回转动力单元以及应用其的机器人.md"
ims_dir = r"..\docs.log\zhuanli_RobotHand\CN202021894937.5-一种结构紧凑的回转动力单元以及应用其的机器人\auto\images"
data_dir = os.path.dirname(markdown_file)
print(bool(Path(markdown_file).is_file())) # True 
print(bool(Path(ims_dir).is_dir()))  # True 

True
True


In [ ]:
# data_dir/
#    + images/        # xx.jpg or xx.png ...
#    + markdown_file  # xxx.md

from IPython.display import display, Markdown, Image
# 可视化 docs -> markdown

docs = []


loader = UnstructuredMarkdownLoader(markdown_file, mode="single") # single   elements
loaded_docs = loader.load()

for i, doc in enumerate(loaded_docs):
    print(f"\n--- Doc {i} ---")
    print(repr(doc.page_content))  # 用 repr 看原始字符
    
    
    # # 提取图片引用（如: ![](images/xxxxx.jpg)）
    # image_paths = re.findall(r"!\[.*?\]\((.+?)\)", doc.page_content)
    # print(f"{image_paths = }")
    # if image_paths:
    #     resolved_images = []
    #     for img_ref in image_paths:
    #         img_ref = img_ref.strip()
    #         print(f"{img_ref = }")
    #         # 尝试解析为相对于 ~auto/  | (data_dir) 目录的路径
    #         if img_ref.startswith("images/"):
    #             img_full = (data_dir / img_ref).resolve()
    #             if img_full.exists():
    #                 resolved_images.append(str(img_full))
    #                 if os.path.exists(str(img_full)):
    #                     print(f"[*] 图片存在: {img_full}")
    #                 else:
    #                     print(f"[WARN] 图片不存在: {img_full}")
    #                 doc.metadata["image_paths"] = resolved_images
    # docs.append(doc)


--- Doc 0 ---
'(19) 中华人民共和国国家知识产权局\n\n(12)实用新型专利\n\n(21)申请号202021894937.5 (22)申请日2020.09.02 (73)专利权人杭州宇树科技有限公司地址310053浙江省杭州市滨江区西兴街道东流路88号1幢306室 (72)发明人王兴兴 (74)专利代理机构浙江翔隆专利事务所(普通合伙）33206代理人许守金\n\n(51) Int.CI. B25J 5/00 (2006.01) B25J 9/10 (2006.01) B62D 57/032 (2006.01)\n\n(54) 实用新型名称\n\n一种结构紧凑的回转动力单元以及应用其的机器人\n\n(57)摘要\n\n本实用新型公开了一种结构紧凑的回转动力单元以及应用其的机器人，属于动力单元以及机器人技术领域。现有回转单元方案，需要在输出端设置磁编码器，但会导致关节输出机构无法做到紧凑。本实用新型一种结构紧凑的回转动力单元在太阳轮设置具有中空腔的回转中心；并把输出端编码器芯片设置在回转中心的中空腔内。本实用新型经过不断探索以及试验，改变现有的输出端磁编码器的设置位置，在太阳轮的回转中心设置中空腔，所述中空腔能够容纳磁编码器的输出端编码器芯片，进而把输出端编码器芯片放置在中空腔中，能够有效减少磁编码器的空间占用，使得回转动力单元结构更加紧凑，结构简单、实用，方案切实可行。\n\n\n\n1.一种结构紧凑的回转动力单元，包括能够提供动能的电机、与电机转子相连接的太阳轮(13)、能够检测转动角度的磁编码器、用于装配电机的基座(17)；其特征在于，\n\n所述太阳轮(13)在回转中心设置中空腔；所述磁编码器包括输出端编码器芯片(41)；所述基座(17)固接一能够穿设太阳轮(13)中空腔的支撑杆(4)；所述支撑杆(4)上装配所述输出端编码器芯片(41)，并连同输出端编码器芯片(41)设置在太阳轮(13)的中空腔内。2.如权利要求1所述的一种结构紧凑的回转动力单元，其特征在于，所述磁编码器设置一能够与输出端编码器芯片(41)磁性感应的输出端磁环(6)；所述太阳轮(13)装配一行星架(7）；所述输出端磁环 (6)与行星架(7)共轴线安装在一起。3.如权利要求2所述的一种结构紧凑的回转动力单元，其特征在于，所述太阳轮 (13)的中空腔靠近输出端磁环(6)的一

In [5]:
for i, doc in enumerate(docs):
    display(Markdown(f"## 📄 Document {i}"))
    display(Markdown(doc.page_content))

    # 如果有图片路径，尝试显示
    if "image_paths" in doc.metadata:
        for img_path in doc.metadata["image_paths"]:
            if Path(img_path).exists():
                display(Image(filename=img_path))